In [ ]:
pip install fuzzywuzzy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Новый раздел

In [4]:
import requests
from bs4 import BeautifulSoup
import tensorflow as tf
import json
import numpy as np
from fuzzywuzzy import fuzz

# links
understat = {
    'match_link': 'https://understat.com/match/',
    '22/23': {
        'EPL': 'https://understat.com/league/EPL/2022',
        'Serie A': 'https://understat.com/league/Serie_A/2022',
        'La liga': 'https://understat.com/league/La_liga/2022',
        'Bundesliga': 'https://understat.com/league/Bundesliga/2022',
        'Ligue 1': 'https://understat.com/league/Ligue_1/2022'
    },
    '21/22': {
        'EPL': 'https://understat.com/league/EPL/2021',
        'Serie A': 'https://understat.com/league/Serie_A/2021',
        'La liga': 'https://understat.com/league/La_liga/2021',
        'Bundesliga': 'https://understat.com/league/Bundesliga/2021',
        'Ligue 1': 'https://understat.com/league/Ligue_1/2021'
    },
    '20/21': {
        'EPL': 'https://understat.com/league/EPL/2020',
        'Serie A': 'https://understat.com/league/Serie_A/2020',
        'La liga': 'https://understat.com/league/La_liga/2020',
        'Bundesliga': 'https://understat.com/league/Bundesliga/2020',
        'Ligue 1': 'https://understat.com/league/Ligue_1/2020'
    },
    '19/20': {
        'EPL': 'https://understat.com/league/EPL/2019',
        'Serie A': 'https://understat.com/league/Serie_A/2019',
        'La liga': 'https://understat.com/league/La_liga/2019',
        'Bundesliga': 'https://understat.com/league/Bundesliga/2019',
        'Ligue 1': 'https://understat.com/league/Ligue_1/2019'
    },
    '18/19': {
        'EPL': 'https://understat.com/league/EPL/2018',
        'Serie A': 'https://understat.com/league/Serie_A/2018',
        'La liga': 'https://understat.com/league/La_liga/2018',
        'Bundesliga': 'https://understat.com/league/Bundesliga/2018',
        'Ligue 1': 'https://understat.com/league/Ligue_1/2018'
    },
    '17/18': {
        'EPL': 'https://understat.com/league/EPL/2018',
        'Serie A': 'https://understat.com/league/Serie_A/2017',
        'La liga': 'https://understat.com/league/La_liga/2017',
        'Bundesliga': 'https://understat.com/league/Bundesliga/2017',
        'Ligue 1': 'https://understat.com/league/Ligue_1/2017'
    },
    '16/17': {
        'EPL': 'https://understat.com/league/EPL/2016',
        'Serie A': 'https://understat.com/league/Serie_A/2016',
        'La liga': 'https://understat.com/league/La_liga/2016',
        'Bundesliga': 'https://understat.com/league/Bundesliga/2016',
        'Ligue 1': 'https://understat.com/league/Ligue_1/2016'
    },
    '15/16': {
        'EPL': 'https://understat.com/league/EPL/2015',
        'Serie A': 'https://understat.com/league/Serie_A/2015',
        'La liga': 'https://understat.com/league/La_liga/2015',
        'Bundesliga': 'https://understat.com/league/Bundesliga/2015',
        'Ligue 1': 'https://understat.com/league/Ligue_1/2015'
    },
    '14/15': {
        'EPL': 'https://understat.com/league/EPL/2014',
        'Serie A': 'https://understat.com/league/Serie_A/2014',
        'La liga': 'https://understat.com/league/La_liga/2014',
        'Bundesliga': 'https://understat.com/league/Bundesliga/2014',
        'Ligue 1': 'https://understat.com/league/Ligue_1/2014'
    },
}
# enums
HOME, AWAY = 0, 1
DRAW, WON, LOST = 0, 1, 2
W_PTS, D_PTS, L_PTS = 3, 1, 0

In [5]:
class Club :
    def __init__(self, club : str) :
        self.last_5, self.last_10, self.last_15, self.last_25, self.last_38 = [], [], [], [], []
        self.club = club

In [6]:
def get_all_clubs(url) :
    clubs = []
    res = requests.get(url)

    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
    strings = scripts[1].string

    ind_start = strings.index("('") + 2
    ind_end = strings.index("')")

    json_data = strings[ind_start : ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')

    data = json.loads(json_data)
    for i in range(10) :
        clubs.append(data[i]['h']['title'])
        clubs.append(data[i]['a']['title'])
    return set(clubs)

all_clubs = []
for key, value in understat.items() :
    if (key == 'match_link') : continue
    for name, link in value.items() :
        all_clubs = all_clubs + list(get_all_clubs(link))
all_clubs = set(all_clubs)

all_club_dict = {}

for club in all_clubs :
    all_club_dict[club] = Club(club)

In [10]:
model = tf.keras.models.load_model('/content/prediction_model')
loaded_clubs = np.load('clubs_stats_for_predictions.npy', allow_pickle=True).item()

for club in all_clubs :
    all_club_dict[club].last_5 = loaded_clubs[club][0][0]
    all_club_dict[club].last_10 = loaded_clubs[club][0][1]
    all_club_dict[club].last_15 = loaded_clubs[club][0][2]
    all_club_dict[club].last_25 = loaded_clubs[club][0][3]
    all_club_dict[club].last_38 = loaded_clubs[club][0][4]



In [11]:
def match_names(name : str) :
        name = name.lower()
        closest_approximation = ""
        greatest_value = -1

        for club in all_clubs :
            ratio = fuzz.ratio(name, club.lower())
            partial_ratio = fuzz.partial_ratio(name, club.lower())

            value = ratio * 0.6 + partial_ratio * 0.4

            if (value > greatest_value) :
                greatest_value = value
                closest_approximation = club

        if (greatest_value < 35) : return None
        else : return closest_approximation

In [12]:
def predict(home_team, away_team) :
    home_team = match_names(home_team)
    away_team = match_names(away_team)
    input = []

    input.append(all_club_dict[home_team].last_5
            + all_club_dict[home_team].last_10
            + all_club_dict[home_team].last_15
            + all_club_dict[home_team].last_25
            + all_club_dict[home_team].last_38
            + all_club_dict[away_team].last_5
            + all_club_dict[away_team].last_10
            + all_club_dict[away_team].last_15
            + all_club_dict[away_team].last_25
            + all_club_dict[away_team].last_38
        )
    prediction = model.predict(input)

    print('DRAW chance:', "{:.2f}".format(prediction[0][0] * 100))
    print(home_team, 'WIN chance:', "{:.2f}".format(prediction[0][1] * 100))
    print(away_team, 'WIN chance:', "{:.2f}".format(prediction[0][2] * 100))

In [13]:
import builtins

print("Football Match Prediction:\n")
pred_home_team = builtins.input('Home Team:')
print('You entered:', match_names(pred_home_team))
pred_away_team = builtins.input('Away Team:')
print('You entered:',  match_names(pred_away_team))
predict(pred_home_team, pred_away_team)

Football Match Prediction:

Home Team:PSG
You entered: Pescara
Away Team:Real Madrid
You entered: Real Madrid
1/1 [==============================] - 0s 187ms/step
DRAW chance: 20.19
Pescara WIN chance: 16.23
Real Madrid WIN chance: 63.58
